In [108]:
# this script is to create an OT dataset
# information record: lat,lon, mintb, citb, heterogeneity, ot flag

def read_text(fname):
    fo=open(fname,'r') #create file object
    #headstr=fo.readline()
    data=fo.readlines()
    #print (data)
    fo.close()  # close object
    return data

import os 
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta
import numpy as np
import pandas as pd
import glob
from pyhdf.SD import SD, SDC
import h5py
import netCDF4 as nc
import statsmodels.api as sm
from scipy import stats
from math import floor
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
#check for land-sea mask
from global_land_mask import globe


print('start',datetime.now())
planck_c1=1.191042e8
planck_c2=1.4387752e4
#window define
win_size=64

fnames=glob.glob('/data/keeling/a/yulanh/mydata/OT_modis/plot_output/hc_record_information_gpm_modis_1505_update_20*')
wotfname=open('/data/keeling/a/yulanh/c/OT_output/OT_check_database/OT_database_manual_check.txt','w')
wotfname.write('OT_index, time, lat, lon, ota, min_BT11, ave_ciBT11, tropopauseT, heterogeneity, land fraction, ls_mask_ot \n')

wnotfname=open('/data/keeling/a/yulanh/c/OT_output/OT_check_database/NOT_database_manual_check.txt','w')
wnotfname.write('NOT_index, time, lat, lon, ota, min_BT11, ave_ciBT11, tropopauseT, heterogeneity, land fraction, ls_mask_ot \n')

OT_indexing=0
NOT_indexing=0

for fname in fnames:
    year = fname[-8:-4]
    print(year,fname)
    data=read_text(fname)
    splitcol=data[0].split(' ')
    Ncol=len(splitcol)-splitcol.count('')
    Nrow=len(data)
    #rint(Ncol,Nrow)

    dataT=np.zeros((Nrow,Ncol-1),'f')

    colocated_fname = np.chararray(Nrow,itemsize=99)

    for i in range(Nrow):
        splitcol=data[i].split(' ')
        k=0
        for j in range(len(splitcol)):
            if (len(splitcol[j]) == 99):
                colocated_fname[i]=splitcol[j]
            
            if len(splitcol[j]) != 0 and splitcol[j] !='\n' and len(splitcol[j]) != 99:
                dataT[i,k]=float(splitcol[j])
                k=k+1
        

    GOTlon=dataT[:,0]
    GOTlat=dataT[:,1]
    OTflag=dataT[:,6] # a flag that from mannually check 0-NOT, 1-OT, 3-night time, 2-likely OT
    Proba =dataT[:,2]
    #only deal with OT and NOT
    ind=np.where(np.logical_or(OTflag == 0, OTflag ==1))[0]
    Totnum=len(ind)
    
    GOTlon_1=GOTlon[ind]
    GOTlat_1=GOTlat[ind]
    Proba_1 = Proba[ind]
    OTflag_1=OTflag[ind]
    colocated_fname_1=colocated_fname[ind]

    for fi in range(Totnum):

    # read the spatial data
        tpfname=colocated_fname_1[fi]
        print(tpfname)
        data=read_text(tpfname)
        splitcol=data[0].split(' ')
        MODNcol=len(splitcol)-splitcol.count('')
        MODNrow=len(data)
        #print(MODNcol,MODNrow)
        # transfer data from string to float
        dataT=np.zeros((MODNrow,MODNcol),'f') 
        for i in range(MODNrow):
            splitcol=data[i].split(' ')
            #print(splitcol)
            k=0
            for j in range(len(splitcol)):
                if splitcol[j] != '' and splitcol[j] != '\n' :
                    dataT[i,k]=float(splitcol[j])
                    k=k+1
        
        MODlat_scp=dataT[:,10]
        MODlon_scp=dataT[:,11]
        MODlon=dataT[:,12] # modis lon&lat
        MODlat=dataT[:,13]
        MODprob=dataT[:,14]
        OTA = dataT[:,15]
        MODCBT11=dataT[:,16] # center
        MODaBT11=dataT[:,17] #average
        MODCBT67=dataT[:,18]
        MODaBT67=dataT[:,19]
        tropopause_T=dataT[:,20]
        MODciaBT=dataT[:,21] #ci ave BT
        MODciBT=dataT[:,22:30]
        OTedge = dataT[:,30:38]
        OTstd  =dataT[:,38]
        Cistd  =dataT[:,39]
        OTCistd  =dataT[:,40]
     
         #obtain the information of OT
        ind=np.where((GOTlon_1[fi] == MODlon) & (GOTlat_1[fi] == MODlat) & (Proba_1[fi] == MODprob))
        ind=ind[0]
        
        tpmodlon_scp= int(MODlon_scp[ind])
        tpmodlat_scp= int(MODlat_scp[ind])
        tpota = OTA[ind]
        tpotavebt11=MODaBT11[ind]
        tpotminbt11=MODCBT11[ind]
        tpotavebt67=MODaBT67[ind]
        tpotminbt67=MODCBT67[ind]
        tptrop_T = tropopause_T[ind]
        tpciavebt11= MODciaBT[ind]
        tpotcistd=OTCistd[ind]
        #hetero_OT[fi]=tpotstd/tpotavebt
        #hetero_Ci[fi]=tpcistd/tpciavebt
        tphetero_OTCi=2*tpotcistd/(tpciavebt11+tpotavebt11)
        # add information to array
        #print(OTflag[fi])
        #print(tpfname,MODlon[ind],tpmodlon_scp,GOTlon_1[fi],MODciaBT)
        Julday=str(tpfname[-12:-9], 'utf-8')
        yyyymmdd=str(tpfname[-16:-4], 'utf-8')
        mod03fname=glob.glob('/data/keeling/a/yulanh/satellite/TerraDataArchive/MODIS/MOD03/'+year+'/'+Julday+'/MOD03.A'+yyyymmdd+'*')
        mod02fname=glob.glob('/data/keeling/a/yulanh/satellite/TerraDataArchive/MODIS/MOD021KM/'+year+'/'+Julday+'/MOD021KM.A'+yyyymmdd+'*')
        #print(yyyymmdd,mod03fname,mod02fname)
        hdf=SD(mod03fname[0])
        modlat_matrix=hdf.select('Latitude')
        modlat_matrix=modlat_matrix[:,:]
        modlon_matrix=hdf.select('Longitude')
        modlon_matrix=modlon_matrix[:,:]
        modlatsea = hdf.select('Land/SeaMask')
        modlatsea = modlatsea[:,:]
        otlsmask = modlatsea[tpmodlon_scp-1,tpmodlat_scp-1]
        #print('check lon lat',modlon_matrix.shape,modlon_matrix[tpmodlon_scp-1,tpmodlat_scp-1],\
        #      modlat_matrix[tpmodlon_scp-1,tpmodlat_scp-1])
        hdf=SD(mod02fname[0],SDC.READ)
        emis_obj=hdf.select('EV_1KM_Emissive')

        for key, value in emis_obj.attributes().items():
            if key == 'radiance_offsets':
                add_offset = value  
            if key == 'radiance_scales':
                scale_factor = value
        emis11=scale_factor[10]*(emis_obj[10,:,:]-add_offset[10])
        bt11=planck_c2/(11.03*np.log(planck_c1/(emis11*11.03**5.0)+1))  
        
    
        emis67=scale_factor[6]*(emis_obj[6,:,:]-add_offset[6])
        bt67=planck_c2/(6.72*np.log(planck_c1/(emis67*6.72**5.0)+1))
        moddim=bt67.shape
        dim_modrow=moddim[0]
        dim_modcol=moddim[1]
        lowscp1=tpmodlon_scp-int(win_size/2)
        upscp1=tpmodlon_scp+int(win_size/2)
        lowscp2=tpmodlat_scp-int(win_size/2)
        upscp2=tpmodlat_scp+int(win_size/2)
        if (lowscp1 < 0):
            lowscp1=0
        if (upscp1 > dim_modrow):
            upscp1=dim_modrow
        if (lowscp2 < 0):
            lowscp2=0
        if (upscp2 > dim_modcol):
            upscp2=dim_modcol
        tpbt11=bt11[lowscp1:upscp1,lowscp2:upscp2]
        tpbt67=bt67[lowscp1:upscp1,lowscp2:upscp2]
        tpmodlon=modlon_matrix[lowscp1:upscp1,lowscp2:upscp2]
        tpmodlat=modlat_matrix[lowscp1:upscp1,lowscp2:upscp2]
        tplandsea= modlatsea[lowscp1:upscp1,lowscp2:upscp2]
        #lsind=np.where((tplandsea == 7) | (tplandsea == 6) | (tplandsea == 0))[0]
        lsind=np.where(tplandsea ==1)[0]
        sea_fraction=len(lsind)/(win_size*win_size)
        
        #print('check bt11,6.7',bt11[tpmodlon_scp-1,tpmodlat_scp-1],bt67[tpmodlon_scp-1,tpmodlat_scp-1])
        # read MERRA2 
           
        date=datetime.strptime(year+'-'+Julday,"%Y-%j").strftime("%Y%m%d")
        hour_scp=round(float(yyyymmdd[8:10])+float(yyyymmdd[10:12])/60.0)-1
   
        merra2_fname='/data/keeling/a/yulanh/c/MERRA-2/'+year+'/MERRA2_400.inst1_2d_asm_Nx.'+date+'.nc4'
        ds=nc.Dataset(merra2_fname)
        trop_t=ds['TROPT'][:]
        merra2_lon=ds['lon'][:]
        merra2_lat=ds['lat'][:]
        
        tptropt=np.zeros((tpmodlon.shape[0],tpmodlon.shape[1]),'f')
        for ai in np.arange(tpmodlon.shape[0]):
            for ci in np.arange(tpmodlon.shape[1]):
                tpmodlat_1=tpmodlat[ai,ci]
                tpmodlon_1=tpmodlon[ai,ci]
                merralon_scp= round((tpmodlon_1+180)/0.625) 
                merralat_scp= round((tpmodlat_1+90)/0.5)
                if (merralon_scp >= len(merra2_lon)):
                    merralon_scp = len(merra2_lon)-1
                if (merralat_scp >= len(merra2_lat)):
                    merralat_scp = len(merra2_lat)-1
                    
                tptropt[ai,ci]=trop_t[hour_scp,merralat_scp,merralon_scp]
        
        #=== plot results ==============
        #p0=plt.pcolormesh(tpmodlon,tpmodlat,tptropt,cmap='jet')
        #cb=plt.colorbar(p0)
        #cb.set_label(label='Tb11 (K)')
        #plt.show()
        
        #===== write results =============================================
        if (OTflag_1[fi] == 1): 
            maxtrix_wfname='/data/keeling/a/yulanh/c/OT_output/OT_check_database/OT_'+str(OT_indexing)+'_'+yyyymmdd+'.h5'
            wotfname.write(str(OT_indexing)+', '+yyyymmdd+', '+str(GOTlon_1[fi])+', '+str(GOTlat_1[fi])+', '+str(tpota[0])+', '+\
                str(tpotminbt11[0])+', '+str(tpciavebt[0])+', '+ str( tptrop_T[0])+' '+str(tphetero_OTCi[0])+', '+\
                str(sea_fraction)+', '+str(otlsmask)+'\n')              
            OT_indexing = OT_indexing+1
            #print('ot',OT_indexing)
           
        if (OTflag_1[fi] == 0):
            maxtrix_wfname='/data/keeling/a/yulanh/c/OT_output/OT_check_database/NOT_'+str(NOT_indexing)+'_'+yyyymmdd+'.h5'
            wnotfname.write(str(NOT_indexing)+', '+yyyymmdd+', '+str(GOTlon_1[fi])+', '+str(GOTlat_1[fi])+', '+str(tpota[0])+', '+\
                          str(tpotminbt11[0])+', '+str(tpciavebt[0])+', '+ str( tptrop_T[0])+', '+str(tphetero_OTCi[0])+', '+\
                           str(sea_fraction)+', '+str(otlsmask)+'\n')                     
            NOT_indexing = NOT_indexing+1
            #print('not',NOT_indexing)
            
        hf=h5py.File(maxtrix_wfname,'w')
        hf.create_dataset('Lon',data=tpmodlon)
        hf.create_dataset('Lat',data=tpmodlat)
        hf.create_dataset('BT11',data= tpbt11)
        hf.create_dataset('BT6.7',data=tpbt67)
        hf.create_dataset('colocated_tropopause_temperature',data= tptropt)
        hf.create_dataset('Land_sea_mask',data=tplandsea)
        hf.close()
        
wotfname.close()
wnotfname.close()

start 2022-07-13 16:04:41.425156
2018 /data/keeling/a/yulanh/mydata/OT_modis/plot_output/hc_record_information_gpm_modis_1505_update_2018.txt
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018008.2335.txt'


/tmp/ipykernel_224162/302830622.py:172: RuntimeWarning: invalid value encountered in log
  bt67=planck_c2/(6.72*np.log(planck_c1/(emis67*6.72**5.0)+1))
/tmp/ipykernel_224162/302830622.py:205: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  trop_t=ds['TROPT'][:]
/tmp/ipykernel_224162/302830622.py:206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  merr

b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018008.2335.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018008.2335.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018011.0240.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018011.0240.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018013.0540.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018013.2205.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018013.2205.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018013.2205.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018014.1935.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_15

b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018099.1635.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018099.1635.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018099.2135.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018099.2135.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018102.0345.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018102.0345.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018102.0345.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018102.0655.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018109.0200.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_15

b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018186.1620.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018187.2015.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018188.0255.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018188.0255.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018188.0255.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018188.0255.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018188.0255.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018189.0340.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_1505_parallax_2018189.0340.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2018/colocated_file_15

...

b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020034.1745.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020034.1745.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020038.2350.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020055.0945.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020055.0945.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020055.2300.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020058.0030.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020058.0030.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020058.0030.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_15

b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020146.1610.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020146.1615.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020146.1615.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020146.1615.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020147.2155.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020148.1600.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020157.2105.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020163.2035.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020182.0255.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_15

b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020189.1750.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020189.1750.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020189.1750.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020189.1750.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020189.1755.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020189.1755.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020189.1755.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020189.1755.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020189.1755.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_15

b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020219.0425.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020219.0425.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020219.0425.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020220.0330.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020220.0330.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020220.0650.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020220.0650.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020222.1135.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020222.1135.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_15

b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020318.2330.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020319.2230.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020319.2230.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020321.2225.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020321.2225.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020353.1225.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020353.1850.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020353.2040.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_1505_parallax_2020353.2040.txt'
b'/data/keeling/a/yulanh/mydata/OT_modis/GPM-Terra/2020/colocated_file_15

64
